In [2]:
%load_ext autoreload
%autoreload 2

import geopandas as gpd
import folium
import folium.plugins
import logging
from eo_tools.util import explore_products
from eo_tools_dev.util import show_s2_color
from eo_tools.S2 import process_s2_tiles


from eodag import EODataAccessGateway

# credentials need to be stored in the following file (see EODAG docs)
confpath = "/data/eodag_config.yml"
dag = EODataAccessGateway(user_conf_file_path=confpath)
# make sure cop_dataspace will be used 
dag.set_preferred_provider("cop_dataspace")
logging.basicConfig(level=logging.INFO)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


INFO:eodag.config:Loading user configuration from: /data/eodag_config.yml
INFO:eodag.core:usgs: provider needing auth for search has been pruned because no crendentials could be found
INFO:eodag.core:aws_eos: provider needing auth for search has been pruned because no crendentials could be found
INFO:eodag.core:meteoblue: provider needing auth for search has been pruned because no crendentials could be found
INFO:eodag.core:hydroweb_next: provider needing auth for search has been pruned because no crendentials could be found
INFO:eodag.core:wekeo: provider needing auth for search has been pruned because no crendentials could be found
INFO:eodag.core:creodias_s3: provider needing auth for search has been pruned because no crendentials could be found
INFO:eodag.core:Locations configuration loaded from /root/.config/eodag/locations.yml


# Directory setup

In [3]:
# change to your data directories
path_data = "/data"
path_s2 = f"{path_data}/S2"
path_processed = f"{path_data}/res"

## Select an area of interest

- Select a drawing tool (polygon, circle or rectangle)
- Draw the AOI on the map
- Click "Export" on the right to save the GeoJSON file

In [ ]:
map_select = folium.Map()
folium.plugins.Draw(
    export=True,
    draw_options={
        "polyline": False,
        "circlemarker": False,
        "marker": False,
    },
).add_to(map_select)
map_select

# Search

In [4]:
# load a geometry
file_aoi = "../data/marketing_aoi.geojson"
shp = gpd.read_file(file_aoi).geometry[0]
search_criteria = {
    "productType": "S2_MSI_L1C",
    "start": "2023-06-01",
    "end": "2023-08-31",
    "cloudCover": 10,
    "geom": shp
}
results = dag.search_all(**search_criteria)

INFO:eodag.core:Searching product type 'S2_MSI_L1C' on provider: cop_dataspace
INFO:eodag.core:Iterate search over multiple pages: page #1
INFO:eodag.search.qssearch:Sending search request: http://catalogue.dataspace.copernicus.eu/resto/api/collections/Sentinel2/search.json?cloudCover=[0,10]&startDate=2023-06-01&completionDate=2023-08-31&geometry=POLYGON ((-10.3160 30.5720, -10.3160 31.3819, -7.9313 31.3819, -7.9313 30.5720, -10.3160 30.5720))&productType=S2MSI1C&maxRecords=1000&page=1&exactCount=1
INFO:eodag.core:Found 146 result(s) on provider 'cop_dataspace'


Getting unique dates to help with product exploration

In [5]:
def get_unique_dates(results):
    import numpy as np

    dates = []
    for p in results:
        dates.append(p.properties["startTimeFromAscendingNode"])
    return np.unique(dates).tolist()

dates = get_unique_dates(results)
print(f"unique dates:")
print(", \n".join(dates))

unique dates:
2023-06-03T11:21:21.024Z, 
2023-06-05T11:06:19.024Z, 
2023-06-10T11:06:21.024Z, 
2023-06-13T11:21:21.025Z, 
2023-06-15T11:06:29.024Z, 
2023-06-18T11:21:19.024Z, 
2023-06-20T11:06:21.024Z, 
2023-06-23T11:21:21.024Z, 
2023-06-25T11:06:19.024Z, 
2023-06-28T11:21:19.024Z, 
2023-06-30T11:06:21.024Z, 
2023-07-03T11:21:21.024Z, 
2023-07-05T11:06:29.024Z, 
2023-07-08T11:21:19.024Z, 
2023-07-10T11:06:21.024Z, 
2023-07-15T11:06:29.024Z, 
2023-07-18T11:21:19.024Z, 
2023-07-20T11:06:21.024Z, 
2023-07-23T11:21:21.024Z, 
2023-07-25T11:06:29.024Z, 
2023-07-28T11:21:19.024Z, 
2023-07-30T11:06:21.024Z, 
2023-08-02T11:21:21.024Z, 
2023-08-04T11:06:19.024Z, 
2023-08-09T11:06:21.024Z, 
2023-08-12T11:21:21.024Z, 
2023-08-14T11:06:29.024Z, 
2023-08-17T11:21:19.024Z, 
2023-08-19T11:06:21.024Z, 
2023-08-22T11:21:21.024Z, 
2023-08-24T11:06:19.024Z, 
2023-08-27T11:21:19.024Z, 
2023-08-29T11:06:21.024Z


# Visualize product footprints on a map

In [17]:
# show all results
# explore_products(results, shp)

# filter by date
results_filtered = results.filter_date("2023-07-01", "2023-07-09") 
explore_products(results_filtered, shp)


INFO:eodag.crunch.date:Finished filtering products. 17 resulting products


# Download

In [20]:
# re-run to get only links
to_download = [results[i] for i in [3,10,6,0,7,8]]
dl = dag.download_all(to_download, outputs_prefix=path_s2)

INFO:eodag.core:Downloading 6 products


Downloaded products:   0%|          | 0/6 [00:00<?, ?product/s]

0.00B [00:00, ?B/s]

INFO:eodag.download.base:Download url: https://catalogue.dataspace.copernicus.eu/odata/v1/Products(2582c0d8-9fd1-4fed-a24b-43f9619d71fd)/$value
INFO:eodag.download.base:Extraction not activated. The product is available as is.
INFO:eodag.product:Remote location of the product is still available through its 'remote_location' property: https://catalogue.dataspace.copernicus.eu/odata/v1/Products(2582c0d8-9fd1-4fed-a24b-43f9619d71fd)/$value


0.00B [00:00, ?B/s]

...

INFO:eodag.download.base:Download url: https://catalogue.dataspace.copernicus.eu/odata/v1/Products(61ebc93f-0f9b-4d44-b013-5a6f5a284898)/$value
INFO:eodag.download.base:Extraction not activated. The product is available as is.
INFO:eodag.product:Remote location of the product is still available through its 'remote_location' property: https://catalogue.dataspace.copernicus.eu/odata/v1/Products(61ebc93f-0f9b-4d44-b013-5a6f5a284898)/$value


0.00B [00:00, ?B/s]

INFO:eodag.download.base:Download url: https://catalogue.dataspace.copernicus.eu/odata/v1/Products(4666daf3-b7b5-4e5e-9eb2-99bbf00f2d52)/$value
INFO:eodag.download.base:Extraction not activated. The product is available as is.
INFO:eodag.product:Remote location of the product is still available through its 'remote_location' property: https://catalogue.dataspace.copernicus.eu/odata/v1/Products(4666daf3-b7b5-4e5e-9eb2-99bbf00f2d52)/$value


0.00B [00:00, ?B/s]

INFO:eodag.download.base:Download url: https://catalogue.dataspace.copernicus.eu/odata/v1/Products(0f6e871b-9a8c-410b-a92a-b05603ca9b72)/$value
INFO:eodag.download.base:Extraction not activated. The product is available as is.
INFO:eodag.product:Remote location of the product is still available through its 'remote_location' property: https://catalogue.dataspace.copernicus.eu/odata/v1/Products(0f6e871b-9a8c-410b-a92a-b05603ca9b72)/$value


# Merge downloaded tiles

In [21]:
out_dirs = process_s2_tiles(
    dl,
    bands=["B2", "B3", "B4", "B8"],
    shp=shp,
    aoi_name="aoi_bretagne",
    outputs_prefix=path_processed,
)

/root/micromamba/envs/eo_tools/lib/python3.12/site-packages/rasterio/__init__.py:304: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
INFO:eo_tools.S2:---- Processing data take GS2A_20230603T112121_041504_N05.09
INFO:eo_tools.S2:--- Processing band 'B2'
INFO:eo_tools.S2:-- Tile /data/S2/S2A_MSIL1C_20230603T112121_N0509_R037_T29RMQ_20230603T164122.zip
/root/micromamba/envs/eo_tools/lib/python3.12/site-packages/rasterio/__init__.py:304: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
INFO:eo_tools.S2:-- Tile /data/S2/S2A_MSIL1C_20230603T112121_N0509_R037_T29RMQ_20230603T150055.zip
/root/micromamba/envs/eo_tools/lib/python3.12/site-packages/rasterio/__init__.py:304: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or

# Display a color representation on a map 

In [27]:
# from folium import Map
# show_s2_color(out_dirs[0], "RGB")

INFO:httpx:HTTP Request: GET http://localhost:8085/cog/info?url=%2Fdata%2Fres%2FS2A_L1C_2023-06-03-112121_aoi_bretagne%2FRGB.tif "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:8085/cog/tilejson.json?url=%2Fdata%2Fres%2FS2A_L1C_2023-06-03-112121_aoi_bretagne%2FRGB.tif&port=8085 "HTTP/1.1 200 OK"
